In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, LeavePOut
from sklearn.neighbors import KNeighborsClassifier

import sys
sys.path.insert(0,'../')
import common.datautils as datautils
import common.featuresutils as featuresutils

In [ ]:
average_data = datautils.load_ce_results('CE.results.zip')
average_data.head()

In [ ]:
benchmarks = average_data["Benchmark"].unique()
benchmarks_nosize = [b.split('.')[0] for b in benchmarks]
print(benchmarks_nosize)

In [ ]:
X, y = featuresutils.load_features(benchmarks_nosize, with_dwarf=False, with_names=True)
X = StandardScaler().fit_transform(X)

### k-Fold Crossvalidation

In [ ]:
kf = KFold(n_splits=20)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    nn = KNeighborsClassifier(n_neighbors=1)
    nn.fit(X_train, y_train)
    prediction = nn.predict(X_test)
    print(y_test)
    print(prediction)
    print("------")
    

### Leave P out Cross Validation

In [ ]:
lpo = LeavePOut(2)
for train_index, test_index in lpo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    nn = KNeighborsClassifier(n_neighbors=1)
    nn.fit(X_train, y_train)
    prediction = nn.predict(X_test)
    print(y_test)
    print(prediction)
    print("------")